# Integrating Key Performance Indicators and Hyperparameters Into SAS Model Manager
SAS Model Manager generates certain Key Performance Indicators (KPIs) automatically based on user created performance definitions. However, we may want to include other KPIs to measure certain things not tracked by SAS Model Manager.

This can be done by performing local tests on models we"ve passed to SAS Model Manager, then passing up the resulting values as custom KPI values.

For certain python models, sasctl will also generate a json file containing the hyperparameters of the model, making them easily accessible for future use.

### Python Package Imports

In [ ]:
# Standard Library
from pathlib import Path
import warnings
import getpass

# Third Party
import pandas as pd
import pickle

# Application Specific
from sasctl import Session
from sasctl.pzmm.model_parameters import ModelParameters as mp

# Global Package Options
pd.options.mode.chained_assignment = None  # default="warn"
warnings.simplefilter(action="ignore", category=FutureWarning)

### Setup
This example is built to be run after the [PZMM Binary Classification Model Import](./pzmm_binary_classification_model_import.ipynb) example notebook is run.

In [ ]:
username = getpass.getpass()
password = getpass.getpass()
host = "demo.sas.com" # Changes required by user
sess = Session(host, username, password, protocol="http") # For TLS-enabled servers, change protocol value to "https"

### Updating Model and Project Properties
In order to allow for performance definitions to be run in SAS Model Manager, certain properties need to be set for both the model and the project.

In [ ]:
from sasctl._services.model_repository import ModelRepository as mr

model_name = "DecisionTreeClassifier"
model_path = Path.cwd() / "data/hmeqModels/DecisionTreeClassifier/"

model = mr.get_model(model_name)

model["targetEvent"] = "1"
model["targetVariable"] = "BAD"
model["function"] = "classification"
model["targetLevel"] = "Binary"
model["eventProbVar"] = "EM_EVENTPROBABILITY"

model = mr.update_model(model)

In [ ]:
project = mr.get_project("HMEQModels")

variables = model["inputVariables"] + model["outputVariables"]

project["targetVariable"] = "BAD"
project["variables"] = variables
project["targetLevel"] = "Binary"
project["targetEventValue"] = "1"
project["classTargetValues"] = ".5"
project["function"] = "classification"
project["eventProbabilityVariable"] = "EM_EVENTPROBABILITY"

project = mr.update_project(project)

### Hyperparameter Generation
If the hyperparameter json file is not generated automatically, this code block will generate it and add it to SAS Model Manager.

In [ ]:
with open(Path(model_path) / "DecisionTreeClassifier.pickle", "rb") as pickle_model:
    tree_model = pickle.load(pickle_model)

mp.generate_hyperparameters(tree_model, model_name, model_path)

with open(model_path / f"{model_name}Hyperparameters.json", "r") as f:
    mr.add_model_content(model_name, f, f"{model_name}Hyperparameters.json")

Once the model has been uploaded to SAS Model Manager, custom hyperparameters can be added to the hyperparameter json file using the add_hyperparamters function. The custom hyperparameters are passed in as kwargs.

In [ ]:
mp.add_hyperparameters(model_name, example=1)

### Performance Definition
To create a performance definition, we first have to pass up data for the performance definition to run on. All data used for performance defintions should be named using the following format: 

{Table Prefix}\_{Time}\_{Time Label}

In [ ]:
from sasctl._services.cas_management import CASManagement as cas

for x in range(1, 5):
    cas.upload_file(
        file=f"data/HMEQPERF_{x}_Q{x}.csv", 
        name=f"HMEQPERF_{x}_Q{x}")
    print(x)

After pushing up the data, the performance definition can be created. When the performance definition is run, the KPIs are generated within Model Manager.

In [ ]:
from sasctl._services.model_management import ModelManagement as mm

perf_task = mm.create_performance_definition(table_prefix="hmeqperf", project="HMEQModels", scoring_required=True)

In [ ]:
project = mr.get_project("HMEQModels")

perf_job = mm.execute_performance_definition(perf_task)

Once the performance definition is run, it is possible to update the hyperparameter json file to include the KPIs that have been generated. This is not a necessary step, but could be helpful when analyzing which hyperparameters lead to better KPIs. Note that you need the performance jobs to have created the MM_STD performance table before running the `update_kpis()` method below.

In [ ]:
check_perf_job = mm.get(f"/performanceTasks/{perf_job[0].taskId}/performanceJobs")
for j in check_perf_job:
    print(f"{j.model}: {j.state}")

In [ ]:
mp.update_kpis("HMEQModels")

### Custom KPIs
It is also possible to generate custom key performance indicators and pass them up to SAS Model Manager. Below, using the same data sets as were used in the SAS performance definition, the recall score is calculated, and then passed up to the KPI table in SAS Model Manager.

In [ ]:
from sklearn.metrics import jaccard_score

predictor_columns = ["LOAN", "MORTDUE", "VALUE", "YOJ", "DEROG", "DELINQ", "CLAGE", "NINQ", "CLNO", "DEBTINC"]
target_column = "BAD"

jaccard_list = list()
time_labels = list()
time_sks = list()
name = ["jaccard" for x in range(4)]

for x in range(1, 5):
    test_data = pd.read_csv(f"data/HMEQPERF_{x}_Q{x}.csv")
    x_test = test_data[predictor_columns]
    y_test = test_data[target_column]
    test_data_predictions = tree_model.predict(x_test)
    jaccard = jaccard_score(y_test, test_data_predictions)
    jaccard_list.append(jaccard)
    time_labels.append(f"Q{x}")
    time_sks.append(x)

print(jaccard_list)

model = mr.get_model(model_name)
mm.create_custom_kpi(
    model=model.id,
    project="HMEQModels",
    kpiName=name,
    kpiValue=jaccard_list,
    timeLabel=time_labels,
    timeSK=time_sks
)

Once the KPIs have been generated, the hyperparameter file can updated, and the new KPIs will appear in the file.

In [ ]:
import json

mp.update_kpis("HMEQModels")

hyperparameters = mp.get_hyperparameters(model_name)

print(json.dumps(hyperparameters, indent=4))